# NER Task
Rishi Gandhi J022

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pii-detection-removal-from-educational-data/sample_submission.csv
/kaggle/input/pii-detection-removal-from-educational-data/train.json
/kaggle/input/pii-detection-removal-from-educational-data/test.json


In [14]:
import pandas as pd

# Load data
train_df = pd.read_json('/kaggle/input/pii-detection-removal-from-educational-data/train.json')
test_df = pd.read_json('/kaggle/input/pii-detection-removal-from-educational-data/test.json')

# Display the first few rows and data types
print("Train DataFrame:")
print(train_df.head())
print(train_df.info())

print("\nTest DataFrame:")
print(test_df.head())
print(test_df.info())


Train DataFrame:
   document                                          full_text  \
0         7  Design Thinking for innovation reflexion-Avril...   
1        10  Diego Estrada\n\nDesign Thinking Assignment\n\...   
2        16  Reporting process\n\nby Gilberto Gamboa\n\nCha...   
3        20  Design Thinking for Innovation\n\nSindy Samaca...   
4        56  Assignment:  Visualization Reflection  Submitt...   

                                              tokens  \
0  [Design, Thinking, for, innovation, reflexion,...   
1  [Diego, Estrada, \n\n, Design, Thinking, Assig...   
2  [Reporting, process, \n\n, by, Gilberto, Gambo...   
3  [Design, Thinking, for, Innovation, \n\n, Sind...   
4  [Assignment, :,   , Visualization,  , Reflecti...   

                                 trailing_whitespace  \
0  [True, True, True, True, False, False, True, F...   
1  [True, False, False, True, True, False, False,...   
2  [True, False, False, True, True, False, False,...   
3  [True, True, True, Fal

In [15]:
# Check if 'labels' column exists and display unique values
if 'labels' in train_df.columns:
    print("\nTrain Labels Sample:")
    print(train_df['labels'].head())
    print("\nUnique Train Labels:")
    print(train_df['labels'].explode().unique())

if 'labels' in test_df.columns:
    print("\nTest Labels Sample:")
    print(test_df['labels'].head())
    print("\nUnique Test Labels:")
    print(test_df['labels'].explode().unique())



Train Labels Sample:
0    [O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-...
1    [B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O...
2    [O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O...
3    [O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT...
4    [O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST...
Name: labels, dtype: object

Unique Train Labels:
['O' 'B-NAME_STUDENT' 'I-NAME_STUDENT' 'B-URL_PERSONAL' 'B-EMAIL'
 'B-ID_NUM' 'I-URL_PERSONAL' 'B-USERNAME' 'B-PHONE_NUM' 'I-PHONE_NUM'
 'B-STREET_ADDRESS' 'I-STREET_ADDRESS' 'I-ID_NUM']


In [16]:
# Display sample text data
print("\nTrain Text Sample:")
print(train_df['full_text'].head())

print("\nTest Text Sample:")
print(test_df['full_text'].head())


# Check for tokenization specifics
sample_text = train_df['full_text'].iloc[0]
print("\nSample Text for Tokenization:")
print(sample_text)



Train Text Sample:
0    Design Thinking for innovation reflexion-Avril...
1    Diego Estrada\n\nDesign Thinking Assignment\n\...
2    Reporting process\n\nby Gilberto Gamboa\n\nCha...
3    Design Thinking for Innovation\n\nSindy Samaca...
4    Assignment:  Visualization Reflection  Submitt...
Name: full_text, dtype: object

Test Text Sample:
0    Design Thinking for innovation reflexion-Avril...
1    Diego Estrada\n\nDesign Thinking Assignment\n\...
2    Reporting process\n\nby Gilberto Gamboa\n\nCha...
3    Design Thinking for Innovation\n\nSindy Samaca...
4    Assignment:  Visualization Reflection  Submitt...
Name: full_text, dtype: object

Sample Text for Tokenization:
Design Thinking for innovation reflexion-Avril 2021-Nathalie Sylla

Challenge & selection

The tool I use to help all stakeholders finding their way through the complexity of a project is the  mind map.

What exactly is a mind map? According to the definition of Buzan T. and Buzan B. (1999, Dessine-moi  l'intelligenc

In [17]:
# Create dummy labels for test data if necessary
if 'labels' not in test_df.columns:
    # Assume each word in the test text should have a dummy label
    test_labels = [[-100] * len(text.split()) for text in test_df['full_text']]
    test_df['labels'] = test_labels

print("\nDummy Test Labels Sample:")
print(test_df['labels'].head())



Dummy Test Labels Sample:
0    [-100, -100, -100, -100, -100, -100, -100, -10...
1    [-100, -100, -100, -100, -100, -100, -100, -10...
2    [-100, -100, -100, -100, -100, -100, -100, -10...
3    [-100, -100, -100, -100, -100, -100, -100, -10...
4    [-100, -100, -100, -100, -100, -100, -100, -10...
Name: labels, dtype: object


# Data Loading:

Load training and test data from JSON files.

# Label Mapping:

Create a mapping from label names to numerical IDs and vice versa.

# Data Conversion:

Convert text labels to numerical IDs and prepare dummy labels for the test data if they don't exist.

# Tokenization:

Tokenize text and align labels with tokens.

# Training Arguments:

Define arguments for training such as the number of epochs, batch size, and logging settings.

# Training:

Initialize the Trainer with the model, arguments, and datasets, then train the model.

# Evaluation:

Evaluate the model's performance and print results.

# Inference:

Save the model, load it for inference, and use a pipeline to make predictions on sample text.

In [1]:
import pandas as pd
from datasets import Dataset
from transformers import DebertaTokenizerFast, DebertaForTokenClassification, Trainer, TrainingArguments
from transformers import pipeline

# Define label mapping
label_list = ['O', 'B-NAME_STUDENT', 'I-NAME_STUDENT', 'B-URL_PERSONAL', 'B-EMAIL', 
              'B-ID_NUM', 'I-URL_PERSONAL', 'B-USERNAME', 'B-PHONE_NUM', 'I-PHONE_NUM', 
              'B-STREET_ADDRESS', 'I-STREET_ADDRESS', 'I-ID_NUM']

label_to_id = {label: i for i, label in enumerate(label_list)}
id_to_label = {i: label for i, label in enumerate(label_list)}

# Load tokenizer and model
model_name = "microsoft/deberta-base"
tokenizer = DebertaTokenizerFast.from_pretrained(model_name)
model = DebertaForTokenClassification.from_pretrained(model_name, num_labels=len(label_to_id))

# Load data
train_df = pd.read_json('/kaggle/input/pii-detection-removal-from-educational-data/train.json')
test_df = pd.read_json('/kaggle/input/pii-detection-removal-from-educational-data/test.json')

# Convert labels to IDs
def convert_labels_to_ids(labels):
    return [label_to_id.get(label, -1) for label in labels]

train_df['labels'] = train_df['labels'].apply(convert_labels_to_ids)

# Handle test data (create dummy labels if necessary)
if 'labels' not in test_df.columns:
    test_labels = [[-100] * len(text.split()) for text in test_df['full_text']]
    test_df['labels'] = test_labels

# Convert DataFrame to HuggingFace Dataset
train_dataset = Dataset.from_pandas(train_df[['full_text', 'labels']])
test_dataset = Dataset.from_pandas(test_df[['full_text', 'labels']])

# Define a function to tokenize and align labels
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples['full_text'], truncation=True, padding="max_length", max_length=128, is_split_into_words=False)
    
    labels = []
    for i, label in enumerate(examples['labels']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        aligned_labels = [-100] * len(tokenized_inputs.input_ids[i])
        for word_id in word_ids:
            if word_id is not None and word_id < len(label):
                aligned_labels[word_id] = label[word_id]
        labels.append(aligned_labels)
    
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Tokenize and align labels
train_dataset = train_dataset.map(tokenize_and_align_labels, batched=True)
test_dataset = test_dataset.map(tokenize_and_align_labels, batched=True)

# Define TrainingArguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # number of training epochs
    per_device_train_batch_size=8,   # batch size for training
    per_device_eval_batch_size=8,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    evaluation_strategy="epoch",     # evaluate model after each epoch
)

# Initialize Trainer
trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,           # evaluation dataset
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()

print("\nEvaluation Results:")
print(results)

# Save the model
model.save_pretrained('./model')
tokenizer.save_pretrained('./model')

# Load the model for inference
model = DebertaForTokenClassification.from_pretrained('./model')
tokenizer = DebertaTokenizerFast.from_pretrained('./model')

# Create a pipeline for NER
nlp_ner = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

# Example prediction
sample_text = "Design Thinking for innovation reflexion-Avril 2021-Nathalie Sylla"
predictions = nlp_ner(sample_text)

print("\nSample Predictions:")
print(predictions)


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/474 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


...

Epoch,Training Loss,Validation Loss
1,0.006100,nan
2,0.002800,nan
3,0.003500,nan


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0


Evaluation Results:
{'eval_loss': nan, 'eval_runtime': 0.127, 'eval_samples_per_second': 78.753, 'eval_steps_per_second': 7.875, 'epoch': 3.0}


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



Sample Predictions:
[{'entity_group': 'LABEL_0', 'score': 0.9483422, 'word': 'Design Thinking for innovation reflexion-Avril', 'start': 0, 'end': 46}, {'entity_group': 'LABEL_2', 'score': 0.4250323, 'word': ' 2021', 'start': 46, 'end': 51}, {'entity_group': 'LABEL_0', 'score': 0.94255066, 'word': '-Nathalie Sylla', 'start': 51, 'end': 66}]


Evaluation Results:

eval_loss: nan

eval_runtime: 0.127 seconds

eval_samples_per_second: 78.753

eval_steps_per_second: 7.875

epoch: 3.0

Sample Predictions:

Entity: LABEL_0, Score: 0.9483422, Word: Design Thinking for innovation reflexion-Avril, Start: 0, End: 46

Entity: LABEL_2, Score: 0.4250323, Word: 2021, Start: 46, End: 51

Entity: LABEL_0, Score: 0.94255066, Word: -Nathalie Sylla, Start: 51, End: 66
